In [1]:
%matplotlib inline
from keras.models import model_from_json  
import matplotlib.pyplot as plt
import numpy as np
import os, random
import cv2 
from PIL import Image 
from keras.preprocessing import image  
from keras.preprocessing.image import img_to_array, load_img 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.applications.vgg16 import preprocess_input, decode_predictions
import keras.backend as K
import glob   
import shutil
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [5]:
#パラメータ設定
batch_size = 16   
file_name = "oaska_all_img"   
label = sorted(glob.glob("oaska_all_img/test")) 
test_dir = "oaska_all_img/test"   
train_dir = "oaska_all_img/train"   

for i in range(len(label)):
    label[i]=label[i].split("/")[-1]  
    print(label[i])    

osaka_A2701_A270302_27000981
osaka_A2701_A270302_27001863
osaka_A2701_A270302_27001881
osaka_A2701_A270302_27004531
osaka_A2701_A270302_27014025
osaka_A2701_A270302_27014228
osaka_A2701_A270302_27049133
osaka_A2701_A270302_27062386
osaka_A2701_A270302_27083645
osaka_A2701_A270302_27089141
osaka_A2701_A270302_27093756
osaka_A2701_A270302_27097422
osaka_A2701_A270302_27097472
osaka_A2701_A270302_27098747
osaka_A2701_A270302_27107382
osaka_A2701_A270302_27117733
osaka_A2701_A270302_27124191


In [6]:
#モデルと重みの読み込み
json_string = open(file_name + ".json").read()   
model = model_from_json(json_string)
model.load_weights(file_name + ".h5") 

In [7]:
def mean_pred(y_true, y_pred):  
    return K.mean(y_pred)

In [8]:
model.compile(optimizer = SGD(lr = 0.0001, momentum = 0.9),  
             loss = "categorical_crossentropy",
              metrics = ["accuracy", mean_pred] 
             )

In [9]:
#generation
test_datagen = ImageDataGenerator(rescale = 1.0/255)

In [10]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (224, 224),
    batch_size = batch_size,
    class_mode = "categorical",
    shuffle = True
)

Found 200 images belonging to 17 classes.


In [11]:
#評価  
score = model.evaluate_generator(test_generator, steps=48)

In [12]:
score   

[0.21276931400581073, 0.9233870967741935, 0.058823528629477304]

In [13]:
print("test_loss:", score[0])
print("test_acc:", score[1])    

test_loss: 0.21276931400581073
test_acc: 0.9233870967741935


In [20]:
filenames = []   
y_true = []   
y_pred = [] 

for l in label:  
    file=glob.glob(test_dir+ "/" + l + "/*")  
    for i in range(len(file)):
        y_true.append(label.index(l))    
    filenames.extend(file)   
    
for file in filenames:   
    temp_img = load_img(file, target_size=(224, 224))  
    #正規化
    temp_img_array = img_to_array(temp_img)   
    temp_img_array = temp_img_array.astype("float32")/255.0   
    temp_img_array = temp_img_array.reshape((1, 224, 224, 3))  
    #予測
    img_pred = model.predict(temp_img_array)
    y_pred.append(np.argmax(img_pred))

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [19]:
acc = accuracy_score(y_true, y_pred)    #正解率
print(acc)

0.925


In [13]:
pre = precision_score(y_true, y_pred, average="macro")     #適合率
print(pre)  

0.8739418819914176


In [14]:
rec = recall_score(y_true, y_pred, average="macro")    #再現率
print(rec)    

0.8914799253034547


In [18]:
f1 = f1_score(y_true, y_pred, average="macro")   #F1値
print(f1)    

SyntaxError: invalid syntax (<ipython-input-18-a5b1d360585e>, line 1)

In [16]:
(2*(rec*pre))/(rec+pre)    #F値

0.8826237904822017